In [28]:
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [39]:
metadata = pd.read_csv("../data/metadata_sample_submission.csv", index_col=[0], parse_dates=["holiday_date"])
bank_holidays_bcn = pd.read_csv("../data/bank_holidays_bcn.csv", index_col=[0])

ValueError: Missing column provided to 'parse_dates': 'holiday_date'

In [38]:
bank_holidays_bcn.info()

<class 'pandas.core.frame.DataFrame'>
Index: 55 entries, 0 to 54
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   holiday_date  55 non-null     object
dtypes: object(1)
memory usage: 880.0+ bytes


In [32]:
metadata

,station_id,month,day,hour,ctx-4,ctx-3,ctx-2,ctx-1
index,,,,,,,,
0,1,1,1,5,0.781481,0.677778,0.696296,0.750000
1,1,1,1,10,0.737374,0.711111,0.711111,0.731624
2,1,1,1,15,0.827778,0.896296,0.901852,0.883333
3,1,1,1,20,0.825926,0.874074,0.927778,0.918519
4,2,1,1,3,0.592593,0.341954,0.275862,0.540230
...,...,...,...,...,...,...,...,...
171897,496,3,31,1,0.828704,0.787037,0.777778,0.853535
171898,496,3,31,6,0.930556,0.944444,0.935185,0.856481
171899,496,3,31,11,0.912037,0.884259,0.518519,0.157407
